<a href="https://colab.research.google.com/github/pedrostanik/projeto_alura_google_gemini/blob/main/projeto_gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-generativeai

In [2]:
from pathlib import Path
import hashlib
import google.generativeai as genai
from google.colab import userdata
import json
import re

import requests
from datetime import datetime
import ast
import time

In [3]:
class WeatherAPI:

    def __init__(self, latitude, longitude):
        self.API_KEY = '88e0bb2abab93d4bde3436cbe10d96df'
        self.LATITUDE = latitude
        self.LONGITUDE = longitude

    def getParamValues(self, current, max, min, sum):
        if current < min:
            min = current
        if current > max:
            max = current
        sum += current
        return max, min, sum

    def execute(self, days_ahead=1):
        import time
        timestamp = int(time.time())

        # Convert timestamp to a datetime object
        dt_object = datetime.fromtimestamp(timestamp)
        current_day = dt_object.date().day

        # Extract date and time components
        date = dt_object.strftime("%Y-%m-%d")
        time = dt_object.strftime("%H:%M:%S")

        base_url = f'https://api.openweathermap.org/data/2.5/forecast?lat={self.LATITUDE}&lon={self.LONGITUDE}&appid={self.API_KEY}'
        response = requests.get(base_url)

        if response.status_code == 200:
            weather_data = response.json()
            weather_data_f = weather_data['list']

            new_weather_param = []
            for three_hours in weather_data_f:
                data_object = ast.literal_eval(str(three_hours))
                timestamp_especific = data_object.get("dt")
                dt_object = datetime.fromtimestamp(int(timestamp_especific))

                # Extracting date and time components
                target_day = dt_object.date().day
                target_hour = dt_object.hour

                three_hours["dt"] = str(date) + str(time)
                new_weather_param.append(three_hours)

            temp_sum = 0
            pres_sum = 0
            wind_sum = 0
            hum_sum = 0
            rain_sum = 0
            counter = 0

            for index, three_hour in enumerate(new_weather_param):

                main = three_hour['main']
                temp_current = main['temp']
                pres_current = main['pressure']
                hum_current = main['humidity']
                wind_current = three_hour['wind']['speed']
                if "rain" in three_hour:
                    rain_current = three_hour['rain']['3h']
                else:
                    rain_current = 0

                #Comparing values
                if index == 0:
                    temp_max = temp_current
                    temp_min = temp_current
                    hum_sum = hum_current
                    hum_max = hum_current
                    hum_min = hum_current
                    wind_max = wind_current
                    wind_min = wind_current
                    pres_max = pres_current
                    pres_min = pres_current
                    rain_max = rain_current
                    rain_min = rain_current

                temp_max, temp_min, temp_sum = self.getParamValues(temp_current, temp_max, temp_min, temp_sum)
                hum_max, hum_min, hum_sum = self.getParamValues(hum_current, hum_max, hum_min, hum_sum)
                wind_max, wind_min, wind_sum = self.getParamValues(wind_current, wind_max, wind_min, wind_sum)
                pres_max, pres_min, pres_sum = self.getParamValues(pres_current, pres_max, pres_min, pres_sum)
                rain_max, rain_min, rain_sum = self.getParamValues(rain_current, rain_max, rain_min, rain_sum)

                counter +=1

            temp_mean = temp_sum/counter
            wind_mean = wind_sum/counter
            pres_mean = pres_sum/counter
            rain_mean = rain_sum/counter
            hum_mean = hum_sum/counter

            retorno = {
                "temp": {"max": rain_max, "min": rain_min, "mean": temp_mean},
                "wind": {"max": wind_max, "min": wind_min, "mean": wind_mean},
                "pres": {"max": pres_max, "min": pres_min, "mean": pres_mean},
                "rain": {"max": rain_max, "min": rain_min, "mean": rain_mean},
                "hum": {"max": hum_max, "min": hum_min, "mean": hum_mean},
            }
            return retorno


        else:
            print("Falha na consulta da API")
            #TODO
            # raise Exception "Falha na consulta da API"

In [4]:
class NaturalDisasterWarning():
  def __init__(self, gemini_model, city, state):
    location_string = f"Me retorne um JSON puro, colocando como chave a string \"LATITUDE\" e \"LONGITUDE\", respectivamente a latitude e longitude, no formato 99.9999, para o município de {city}/{state}"
    response = gemini_model.generate_content(location_string)
    json_content = re.search(r'{.*}', response.text, re.DOTALL).group()
    json_content = json.loads(json_content)
    self.latitude = json_content["LATITUDE"]
    self.longitude = json_content["LONGITUDE"]
    wapi = WeatherAPI(self.latitude, self.longitude)
    self.weather_prediction = wapi.execute()
    self.temperature_mean = self.weather_prediction["temp"]["mean"]
    self.temperature_max = self.weather_prediction["temp"]["max"]
    self.temperature_min = self.weather_prediction["temp"]["min"]
    self.wind_mean = self.weather_prediction["wind"]["mean"]
    self.wind_max = self.weather_prediction["wind"]["max"]
    self.wind_min = self.weather_prediction["wind"]["min"]
    self.pressure_mean = self.weather_prediction["pres"]["mean"]
    self.pressure_max = self.weather_prediction["pres"]["max"]
    self.pressure_min = self.weather_prediction["pres"]["min"]
    self.rain_mean = self.weather_prediction["rain"]["mean"]
    self.rain_max = self.weather_prediction["rain"]["max"]
    self.rain_min = self.weather_prediction["rain"]["min"]
    self.humidity_mean = self.weather_prediction["hum"]["mean"]
    self.humidity_max = self.weather_prediction["hum"]["max"]
    self.humidity_min = self.weather_prediction["hum"]["min"]

    self.general_problemas = f"Quais os problemas que a cidade de {city}, no estado {state}, geralmente têm, em relação à desastres naturais?"


  def describe_weather_warning(self, gemini_model):
    general_problems_response = gemini_model.generate_content(self.general_problemas)
    general_problems = general_problems_response.text

    convo = gemini_model.start_chat(history=[
      {
        "role": "user",
        "parts": [self.general_problemas]
      },
      {
        "role": "model",
        "parts": [general_problems]
      },
    ])

    self.historical_issues = f"Analise esse texto: {general_problems}. Após a análise e levando em conta o histórico de desastres ou problemas de ordem natural nas últimas semanas, meses, ou anos para esse município/região, E sabendo que a previsão do tempo para os próximos 5 dias tem como (temperatura em kelvin, pressão em hPa, umidade em percentual, vento em m/s e chuva em milímetro) temperatura média igual à {self.temperature_mean}, temperatura máxima igual à {self.temperature_max}, temperatura mínima igual à {self.temperature_min}, vento médio igual à {self.wind_mean}, vento máxima igual à {self.wind_max}, vento mínimo igual à {self.wind_min}, pressão média máxima igual à {self.pressure_mean}, pressão máxima igual à {self.pressure_max}, pressão mínima igual à {self.pressure_min}, umidade média igual à {self.humidity_mean}, umidade máxima igual à {self.humidity_max}, umidade mínima igual à {self.humidity_min}. Crie um relato classificando o tipo de alerta em que essa região está, de 0 a 5. Onde 0 é completamente fora de quaiquers riscos, e 5 com um risco real e iminente de ocorrer um desastre de ordem natural."

    time.sleep(5)
    convo.send_message(self.historical_issues)
    print(convo.last.text)
    return convo.last.text


In [5]:
class EscapeRoute():
  def __init__(self, city, state):
    self.question = f"Crie planos de fuga, pensando em desastres naturais que tem relação com o contexto da cidade de {city}, do estado {state}, como por exemplo enchentes, demoronamentos, etc."

  def describe_escape_route(self, gemini_model):
    print(f'question: {self.question}')
    response = gemini_model.generate_content(self.question)
    escape_route = response.text
    print(escape_route)
    return escape_route

In [20]:
class EconomicConsequences():
  def __init__(self, city, state):
    self.question = f"Quais os impactos econômicos se um desastre atingir {city}, no estado de {state}?"

  def describe_consequences(self, gemini_model):
    print(f'question: {self.question}')
    response = gemini_model.generate_content(self.question)
    consequences = response.text
    print(consequences)
    return consequences

In [6]:
class Mitigation():
  def __init__(self, city, state):
    self.question = f"Levando em conta o clima e os problemas e eventos ambientais extremos que ocorrem na {city}, no estado de {state}. Quais melhores iniciativas ambientais para mitigar ou resolver, em prol de evitar desastres naturais?"

  def describe_mitigation(self, gemini_model):
    print(f'question: {self.question}')
    response = gemini_model.generate_content(self.question)
    mitigation = response.text
    print(mitigation)

In [9]:
class Main():

  def __init__(self):
    api_key = userdata.get("API_KEY")
    genai.configure(api_key=api_key)
    generation_config = {
      "temperature": 0.0,
      "top_p": 0.95,
      "top_k": 0,
      "max_output_tokens": 8192,
      "candidate_count": 1
    }
    safety_settings = [
    {
      "category": "HARM_CATEGORY_HARASSMENT",
      "threshold": "BLOCK_ONLY_HIGH"
    },
    {
      "category": "HARM_CATEGORY_HATE_SPEECH",
      "threshold": "BLOCK_ONLY_HIGH"
    },
    {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "threshold": "BLOCK_ONLY_HIGH"
    },
    {
      "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
      "threshold": "BLOCK_ONLY_HIGH"
    },
    ]
    self.model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              safety_settings=safety_settings)

  def init_system(self):
    print(" ----------- BEM-VINDA(O) AO SISTEMA DE GERENCIAMENTO DE DESASTRES NATURAIS! ------------ ")
    prompt = 1
    while prompt != 0:
      choice = int(input("Escolha abaixo o que quer saber:\n " +
                      "(1) Previsão de desastres naturais em minha localidade\n " +
                      "(2) Rotas de fuga e logística de manutenção para casos de desastres iminentes\n " +
                      "(3) Impactos econômicos de um desatre ambiental em minha localidade\n " +
                      "(4) Mitigação e prevenção de impactos ambientais\n " +
                      "(0) Para sair\n"))

      if (choice==1):
        city = str(input("Qual o nome da *cidade* onde está?\n"))
        state = str(input("Qual o nome da *estado* onde está?\n"))
        # additional_info = str(input("Digite alguma informação complementar que queira, como algum evento que aconteceu recentemente ou um histórico, ou qualquer coisa do tipo\n"))
        ndp = NaturalDisasterWarning(self.model, city, state)
        ndp.describe_weather_warning(self.model)
      elif (choice==2):
        city = str(input("Qual o nome da *cidade* onde está?\n"))
        state = str(input("Qual o nome da *estado* onde está?\n"))
        er = EscapeRoute(city, state)
        er.describe_escape_route(self.model)
      elif (choice==3):
        city = str(input("Qual o nome da *cidade* onde está?\n"))
        state = str(input("Qual o nome da *estado* onde está?\n"))
        sec = EconomicConsequences(city, state)
        sec.describe_consequences(self.model)
      elif (choice==4):
        city = str(input("Qual o nome da *cidade* onde está?\n"))
        state = str(input("Qual o nome da *estado* onde está?\n"))
        mit = Mitigation(city, state)
        mit.describe_mitigation(self.model)
      elif (choice==0):
        break
      else:
        print("Por favor, escolha uma das opções acima ou aperte 0 para sair do sistema.")
      # response = chat.send_message(prompt)
      # print("Resposta: " + response.text)


In [10]:
main = Main()
main.init_system()

 ----------- BEM-VINDA(O) AO SISTEMA DE GERENCIAMENTO DE DESASTRES NATURAIS! ------------ 
Escolha abaixo o que quer saber:
 (1) Previsão de desastres naturais em minha localidade
 (2) Rotas de fuga e logística de manutenção para casos de desastres iminentes
 (3) Impactos econômicos de um desatre ambiental em minha localidade
 (4) Mitigação e prevenção de impactos ambientais
 (0) Para sair
0
